In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import vgg16
from torchvision import models

# 定义数据预处理操作
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
#     transforms.RandomCrop(32, padding=4),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 加载 CIFAR-10 数据集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [18]:
def lr_scheduler(epoch):
    if epoch < 100:
        return 0.01
    elif epoch < 200:
        return 0.001
    return 0.0001

In [19]:
# 定义 VGG 模型
model = vgg16(pretrained=True)
model.classifier[6] = nn.Linear(4096, 10)
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4) # 下次测试一下 weight_decay=5e-4
# lr_scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=[lr_scheduler])
lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=400)
# 训练模型
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [20]:
# 定义辅助函数计算正确率
def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    accuracy = correct / total * 100
    return accuracy

In [21]:
best_acc = .9

In [22]:
# 开始训练
for epoch in range(300):

    running_loss = 0.0
    train_accuracy = 0.0
    s = 0
    acc = 0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        train_accuracy = calculate_accuracy(outputs, labels)
        _, predicted = torch.max(outputs, 1)
        acc += (predicted == labels).sum().item()
        s += len(outputs)
        if (i+1) % 20 == 0:
            print('Epoch=%d Step[%3d/%d] loss: %.3f, train accuracy: %.2f %%' %
              (epoch + 1, i + 1, len(trainloader), loss.item(), train_accuracy))
    print("Epoch = %d, Trainning accuracy = %.2f%%"%(epoch+1, acc / s * 100))
    test_accuracy = 0.0
    total_test_samples = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            test_accuracy += calculate_accuracy(outputs, labels)
            total_test_samples += labels.size(0)
    print('Epoch = %d, Testing accuracy: %.2f %%' % (epoch + 1, test_accuracy / total_test_samples * 100))
    lr_scheduler.step()
    if (test_accuracy / total_test_samples * 100) > best_acc:
        best_acc = test_accuracy / total_test_samples * 100
        torch.save(model.state_dict(), f"vgg-224-cifar10-{best_acc:.2f}.pth")

RuntimeError: CUDA out of memory. Tried to allocate 784.00 MiB (GPU 1; 31.75 GiB total capacity; 4.52 GiB already allocated; 470.19 MiB free; 4.53 GiB reserved in total by PyTorch)

In [ ]:
# torch.save(model.state_dict(), "vgg-224-cifar10.pth")

In [ ]:
best_acc = test_accuracy / total_test_samples * 100

In [ ]:
best_acc

In [ ]:
test_accuracy = 0.0
total_test_samples = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_accuracy += calculate_accuracy(outputs, labels)
        total_test_samples += labels.size(0)
print('Epoch = %d, Testing accuracy: %.2f %%' % (epoch + 1, test_accuracy / total_test_samples * 100))

In [1]:
import torch 
from torchvision import models
from torch import nn
from torchvision.transforms import transforms
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [3]:
net = models.vgg16()
net.classifier[6] = nn.Linear(4096, 10)
net.load_state_dict(torch.load('vgg-224-cifar10-93.92.pth'))
net = net.to(device)

In [4]:

cifar_tran_train = transforms.Compose([
#         transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
cifar_tran_test = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=cifar_tran_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=cifar_tran_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
# 定义辅助函数计算正确率
def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    accuracy = correct / total * 100
    return accuracy
test_accuracy = 0.0
total_test_samples = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        test_accuracy += calculate_accuracy(outputs, labels)
        total_test_samples += labels.size(0)
print('Epoch = %d, Testing accuracy: %.2f %%' % (epoch + 1, test_accuracy / total_test_samples * 100))

NameError: name 'epoch' is not defined

In [6]:
test_accuracy / total_test_samples * 100

91.5

In [16]:
def get_split_dataset(dset_name, batch_size, n_worker, val_size, data_root='../data',
                      use_real_val=False, shuffle=True):
    '''
        split the train set into train / val for rl search
    '''
    if shuffle:
        index_sampler = SubsetRandomSampler
    else:  # every time we use the same order for the split subset
        class SubsetSequentialSampler(SubsetRandomSampler):
            def __iter__(self):
                return (self.indices[i] for i in torch.arange(len(self.indices)).int())
        index_sampler = SubsetSequentialSampler

    print('=> Preparing data: {}...'.format(dset_name))
    if dset_name == 'cifar10':
        transform_train = transforms.Compose([
            # transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])
        transform_test = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])
        trainset = torchvision.datasets.CIFAR10(root=data_root, train=True, download=False, transform=transform_train)
        if use_real_val:  # split the actual val set
            valset = torchvision.datasets.CIFAR10(root=data_root, train=False, download=False, transform=transform_test)
            n_val = len(valset)
            assert val_size < n_val
            indices = list(range(n_val))
            np.random.shuffle(indices)
            _, val_idx = indices[val_size:], indices[:val_size]
            train_idx = list(range(len(trainset)))  # all train set for train
        else:  # split the train set
            valset = torchvision.datasets.CIFAR10(root=data_root, train=True, download=False, transform=transform_test)
            n_train = len(trainset)
            indices = list(range(n_train))
            # now shuffle the indices
            np.random.shuffle(indices)
            assert val_size < n_train
            train_idx, val_idx = indices[val_size:], indices[:val_size]

        train_sampler = index_sampler(train_idx)
        val_sampler = index_sampler(val_idx)

        train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=False, sampler=train_sampler,
                                                   num_workers=n_worker, pin_memory=True)
        val_loader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, sampler=val_sampler,
                                                 num_workers=n_worker, pin_memory=True)
        n_class = 10
    return train_loader, val_loader, n_class
train_loader,val_loader, n_class = get_split_dataset('cifar10', 50,1, 5000,data_root='./data',use_real_val=True,shuffle=False)  # same sampling
print(next(iter(train_loader)))

=> Preparing data: cifar10...
[tensor([[[[-1.2854, -1.2854, -1.2854,  ...,  0.4399,  0.4399,  0.4399],
          [-1.2854, -1.2854, -1.2854,  ...,  0.4399,  0.4399,  0.4399],
          [-1.2854, -1.2854, -1.2854,  ...,  0.4399,  0.4399,  0.4399],
          ...,
          [ 1.0021,  1.0021,  1.0021,  ..., -0.0447, -0.0447, -0.0447],
          [ 1.0021,  1.0021,  1.0021,  ..., -0.0447, -0.0447, -0.0447],
          [ 1.0021,  1.0021,  1.0021,  ..., -0.0447, -0.0447, -0.0447]],

         [[-1.1989, -1.1989, -1.1989,  ...,  0.0204,  0.0204,  0.0204],
          [-1.1989, -1.1989, -1.1989,  ...,  0.0204,  0.0204,  0.0204],
          [-1.1989, -1.1989, -1.1989,  ...,  0.0204,  0.0204,  0.0204],
          ...,
          [ 0.4138,  0.4138,  0.4138,  ..., -0.6089, -0.6089, -0.6089],
          [ 0.4138,  0.4138,  0.4138,  ..., -0.6089, -0.6089, -0.6089],
          [ 0.4138,  0.4138,  0.4138,  ..., -0.6089, -0.6089, -0.6089]],

         [[-0.9922, -0.9922, -0.9922,  ..., -0.2118, -0.2118, -0.2118],

In [4]:
len(train_loader)

1000

In [5]:
iter(train_loader)

In [8]:
next(iter(train_loader)

SyntaxError: unexpected EOF while parsing (1960060440.py, line 1)

In [15]:
for i, (input, target) in enumerate(train_loader):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27